# class_probabilities

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sites.bookcave import bookcave

In [ ]:
_, Y, categories, levels, \
_, _, _, _, categories_df \
= bookcave.get_data({'text'}, text_input='filename', only_categories={0, 1, 2, 3, 4, 5, 6}, return_meta=True)

In [ ]:
len(Y)

In [ ]:
categories

In [ ]:
levels

In [ ]:
def get_flat_levels():
    for category_levels in levels:
        for level in category_levels:
            yield level

flat_levels = list(get_flat_levels())
for i, flat_level in enumerate(flat_levels):
    print('{:2d}: {}'.format(i, flat_level))

Encode labels as one-hot vectors.

In [ ]:
enc = OneHotEncoder(categories='auto', sparse=False)
Y_hot = enc.fit_transform(Y)
n, m = Y_hot.shape
corr = np.zeros((m, m))
for given_index in range(m):
    given_match_indices = [i for i, y in enumerate(Y_hot) if y[given_index] == 1]
    for other_index in range(m):
        other_match_indices = [i for i in given_match_indices if Y_hot[i, other_index] == 1]
        corr[other_index, given_index] = len(other_match_indices) / len(given_match_indices)

In [ ]:
corr_df = pd.DataFrame(corr)
ax = sns.heatmap(corr_df, cmap='Blues_r')
category_indices = [4, 8, 10, 14, 18, 22]
ax.hlines(category_indices, *ax.get_xlim(), colors='white')
ax.vlines(category_indices, *ax.get_ylim(), colors='white')
plt.title('Co-occurrence Probabilities for Maturity Rating Levels')
plt.xlabel('Given Rating Level')
plt.ylabel('Probability of Other Rating Level')
fname = os.path.join('..', 'figures', 'level_conditional_probabilities-{:d}.png'.format(len(Y)))
plt.savefig(fname, bbox_inches='tight')
plt.show()

In [ ]:
corr[9, 3]

In [ ]:
corr[9, 16]

In [ ]:
corr[9, 17]

In [ ]:
corr[21, 17]